# Learning Goals
The goal of this assignment is to use the knowledge gained in the course to develop an end-to-end SNN for MNIST digits classification trained using state-of-the-art gradient-descent algorithm. Along the way, you will also learn the basics of developing any machine learning application- how to handle data using data loaders; defining and optimizing loss; evaluating an algorithm using validation set; speeding up training using GPUs. You will also learn the basics of programming using PyTorch which is by far the most widely used library for machine learning research- being used in applications such as autonomous driving, robotic control, cancer research, and much more!

Let's import all the libraries required for this assignment. 

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, sampler
import torch.optim as optim
import os
import time
import pickle
import numpy as np

In [2]:
CUDA_LAUNCH_BLOCKING=1

# Question 1: Limitation of backprop for SNN

## 1a. 
Sketch out the algorithm for training an SNN using backpropagation. Your algorithm should describe when and how the weights are updated. What is the main limitation of using backpropagation for training an SNN? Describe a solution to resolve the limitation. 

## Answer 1a.
The LIF neuron is one of the simplest models used for describing dynamics of spiking neurons. Since the states of LIF neurons can be updated asynchronously based solely on the timing of input events (i.e., without timestepped integration), LIF is computationally efficient. <br>

Neural networks are typically optimized by SGD, meaning that the vector of network parameters or weights θ is moved in the direction of the negative gradient of some loss function L according to $ θ = θ − η∂L/∂θ $, where η is the learning rate. The backpropagation algorithm uses the chain rule to compute the partial derivatives $ ∂L/∂θ $. For completeness we provide here a summary of backprop for conventional fully-connected deep neural networks: <br>
<br>
1. Propagation inputs in the forward direction to compute the pre-activations $(z^{(l)}) $ and activations $ (a^{(l)} = f^{(l)}(z^{(l)})) $ for all the layers up to the output layer $ l_{nl} $, where f is the transfer function of units.
<br>

2. Calculate the error at the output layer: <br>
<center> $ δ^{(nl)} = \frac{∂L(a^{(nl)},y)}{∂z^{(nl)}} = \frac{∂L(a^{(nl)},y)}{∂a^{(nl)}} . f'(z^{nl}) $ </center> <br>
where $ y $ is the label vector indicating the desired output activation
 and $ · $ is element-wise multiplication. <br>

3. Backpropagate the error to lower layers $ l = nl − 1, nl − 2, …, 2: $ <br>
<center> $ δ^{(l)}=((W^{(l)})^{T} δ^{(l + 1)}) ⋅ f′(z^{(l)}) $ </center> <br>
where $ W^{(l)} $ is the weight matrix of the layer $ l. $ <br>

4. Compute the partial derivatives for the update: <br>
<center> $ ∇_{W^{(l)}}L = δ^{(l + 1)}(a^{(l)})^T $ <br>
         $ ∇_{b^{(l)}}L = δ^{(l + 1)} $ </center> <br>
where $ b^{(l)} $ is the bias vector of the layer $ l $. <br>

5. Update the parameters: <br>
<center> $ W^{(l)} = W^{(l)} − η ∇_{W^{(l)}} L $ <br>
         $ b^{(l)} = b^{(l)} − η ∇_{b^{(l)}} L $ <br>

After the forward propagation, the loss function is measured as a difference between target labels and outputs predicted by the network. Then, the gradients of the loss function are estimated at the final layer. The gradients are propagated backward all the way down to the input layer through the hidden layers using recursive chain rule. <br>

The main limitation of using backpropagation in Spiking Neural Networks is training such networks is difficult due to the non-differentiable nature of spike events. <br>

We can overcome this by introducing a novel technique, which treats the membrane potentials of spiking neurons as differentiable signals, where discontinuities at spike times are considered as noise. This enables an error backpropagation mechanism for deep SNNs that follows the same principles as in conventional deep networks, but works directly on spike signals and membrane potentials. Compared with previous methods relying on indirect training and conversion, our technique has the potential to capture the statistics of spikes more precisely.

## 1b. 
In this exercise, we will implement the solution for overcoming backprop limitation for SNN. First the preliminaries: In PyTorch, arrays are called tensors. Gradients are computed automatically using the automatic differentiation package (autograd). The main elements of an autograd function are the forward and backward functions. The forward function simply performs the forward pass, i.e. computing output tensors from the input tensors. The backward function receives the gradient of the output tensors w.r.t. some scalar value, and computes the gradient of the input tensors w.r.t. the same scalar value. 

Below, we define the autograd function class for pseudo-gradient using the rectangular function. Most of the implementation is already written for you. Your task is to fill two key components- i) in the forward function, write the implementation for generating spike outputs from the inputs; ii) in the backward function, write the implementation for computing the gradient of the spike using rectangular psuedo-grad function. 

In [3]:
class PseudoSpikeRect(torch.autograd.Function):
    """ Rectangular Pseudo-grad function """

    @staticmethod
    def forward(ctx, input, vth, grad_win, grad_amp):
        """
        Args:
            input (Torch Tensor): Input tensor containing voltages of neurons in a layer
            vth (Float): Voltage threshold for spiking 
            grad_win (Float): Window for computing pseudogradient
            grad_amp (Float): Amplification factor for the gradients
        
        Returns:
            output (Torch Tensor): Generated spikes for the input
        
        Write the operation for computing the output spikes from the input. The operation should be vectorized, i.e. no loops. 
        """
        
        #Saving variables for backward pass. Nothing to do here
        ctx.save_for_backward(input)
        ctx.vth = vth
        ctx.grad_win = grad_win
        ctx.grad_amp = grad_amp
        
        #Compute output from the input. No loops. Hint: Use Pytorch "greater than" function. 
        output = torch.gt(input, vth).float()

        return output

    @staticmethod
    def backward(ctx, grad_output):
        """
        Args:
            grad_output (Torch Tensor): Gradient of the output
        
        Returns:
            grad (Torch Tensor): Gradient of the input
        
        Write the operation for computing the output spikes from the input. The operation should be vectorized, i.e. no loops. 
        """
        
        #Retrieving variables from forward pass. Nothing to do here. 
        input, = ctx.saved_tensors
        vth = ctx.vth
        grad_win = ctx.grad_win
        grad_amp = ctx.grad_amp
        grad_input = grad_output.clone()
        
        V_minus_Vth = torch.abs(torch.sub(input, vth))
        spike_pseudo_grad = torch.lt(V_minus_Vth, grad_win).byte()

        #Multiplying by gradient amplifier. Nothing to do here
        grad = grad_amp * grad_input * spike_pseudo_grad
        return grad, None, None, None

# Question 2: Creating a Layer

In this exercise, we will create the class definition for a layer of LIF neurons (similar to the implementation in Assignment 2).

Below is the class definition of an LIF neuron layer. Your task is to write the operation for integrating the presynaptic spikes into voltage, and then transforming to spikes. 

In [4]:
class LinearIFCell(nn.Module):
    """ Leaky Integrate-and-fire neuron layer"""

    def __init__(self, psp_func, pseudo_grad_ops, param):
        """
        Args:
            psp_func (Torch Function): Pre-synaptic function
            pseudo_grad_ops (Torch Function): Pseudo-grad function
            param (tuple): Cell parameters (Voltage Threshold, gradient window, gradient amplitude)
        
        This function is complete. You do not need to do anything here. 
        """
        super(LinearIFCell, self).__init__()
        self.psp_func = psp_func
        self.pseudo_grad_ops = pseudo_grad_ops
        self.vdecay, self.vth, self.grad_win, self.grad_amp = param

    def forward(self, input_data, state):
        """
        Forward function
        Args:
            input_data (Tensor): input spike from pre-synaptic neurons
            state (tuple): output spike of last timestep and voltage of last timestep
        Returns:
            output: output spike
            state: updated neuron states
        
        Write the operation for integrating the presynaptic spikes into voltage.
        """
        pre_spike, pre_volt = state
        
        input_data = self.psp_func.forward(input_data)

        #Compute the voltage from the presynaptic inputs. This should be a vectorized operation. No loops. 
        volt = pre_volt * self.vdecay * (1 - pre_spike) + input_data
        
        #Compute the spike output by using the pseudo_grad_ops function. This should be a vectorized operation. No loops.
        output = self.pseudo_grad_ops(input_data, self.vth, self.grad_win, self.grad_amp)

        return output, (output, volt)

# Question 3: Creating a Network 

## 3a.
We will now create a one-layer SNN using the class definitions above. Preliminaries: In Assignment 2, the psp was computed using numpy matrix multiplication of weights and inputs. In PyTorch, nn.Linear() achieves the same. You can find the documentation here: https://pytorch.org/docs/stable/generated/torch.nn.Linear.html. We will use this class to serve as our psp function required for creating a layer according to the implementation in Q2. 

Below is the class definition of a network. Your task is to fill in the required components in the init and forward functions. 

In [5]:
class SingleHiddenLayerSNN(nn.Module):
    """ SNN with single hidden layer """

    def __init__(self, input_dim, output_dim, hidden_dim, param_dict):
        """
        Args:
            input_dim (int): input dimension
            output_dim (int): output dimension
            hidden_dim (int): hidden layer dimension
            param_dict (dict): neuron parameter dictionary for each layer (Voltage Threshold, gradient window, gradient amplitude)
        
        Create hidden and output layers using implementation of the layer in Q2. and using nn.Linear as the psp function. 
        """
        super(SingleHiddenLayerSNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        pseudo_grad_ops = PseudoSpikeRect.apply
        
        #Create the hidden layer. Assume that the hidden layer neuron parameters are in param_dict['hid_layer']. Set bias=False for nn.Linear.  
        m = nn.Linear(self.input_dim, self.hidden_dim, bias=False)
        self.hidden_cell = LinearIFCell(m, pseudo_grad_ops, param_dict['hid_layer'])
        
        #Create the output layer. Output layer params are in param_dict['out_layer']. Set bias=False for nn.Linear.   
        n = nn.Linear(self.hidden_dim, self.output_dim, bias=False)
        self.output_cell = LinearIFCell(n, pseudo_grad_ops, param_dict['out_layer'])
        

    def forward(self, spike_data, init_states_dict, batch_size, spike_ts):
        """
        Forward function
        Args:
            spike_data (Tensor): spike data input (batch_size, input_dim, spike_ts)
            init_states_dict (dict): initial states for each layer- 'hid_layer' for hidden layer; 'out_layer' for output layer. 
            batch_size (int): batch size
            spike_ts (int): spike timesteps
        Returns:
            output: number of spikes of output layer
        
        Write the operations for propagating the input through the network and computing the spike outputs. 
        """
        hidden_state, out_state = init_states_dict['hid_layer'], init_states_dict['out_layer']
        spike_data_flatten = spike_data.view(batch_size, self.input_dim, spike_ts)
        output_list = [] #List to store the output at each timestep
        for tt in range(spike_ts):
            #Retrieve the input at time tt
            ip = spike_data_flatten[:,:,tt]

            #Propagate through the hidden layer
            hidden_output, hidden_state = self.hidden_cell.forward(ip, hidden_state)
            
            #Propagate through the output layer
            output, out_state = self.output_cell.forward(hidden_output, out_state)
            
            #Append output spikes to output list
            output_list.append(output)
        
        #Sum the outputs to compute spike count for each output neuron. Torch.stack and Torch.sum might be useful here. No loops
        stack = torch.stack(output_list)
        output = torch.sum(stack, dim=0)

        return output

## 3b. 
Next, we need a Wrapper class that: i) Initializes the parameters required for creating the SNN class object; ii) creates the SNN class objects using the initial parameters; iii) Computes the SNN output and returns it. The class is already written for you. You do not need to do anything here. Just understand the implementation so that you can use it later. 

In [6]:
class WrapSNN(nn.Module):
    """ Wrapper of SNN """

    def __init__(self, input_dim, output_dim, hidden_dim, param_dict, device):
        """
        Args:
            input_dim (int): input dimension
            output_dim (int): output dimension
            hidden_dim (int): hidden layer dimension
            param_dict (dict): neuron parameter dictionary
            device (device): device
        """
        super(WrapSNN, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        self.device = device
        self.snn = SingleHiddenLayerSNN(input_dim, output_dim, hidden_dim, param_dict)

    def forward(self, spike_data):
        """
        Forward function
        Args:
            spike_data (Tensor): spike data input
        Returns:
            output: number of spikes of output layer
        """
        batch_size = spike_data.shape[0]
        spike_ts = spike_data.shape[-1]
        init_states_dict = {}
        # Hidden layer
        hidden_volt = torch.zeros(batch_size, self.hidden_dim, device=self.device)
        hidden_spike = torch.zeros(batch_size, self.hidden_dim, device=self.device)
        init_states_dict['hid_layer'] = (hidden_spike, hidden_volt)
        # Output layer
        out_volt = torch.zeros(batch_size, self.output_dim, device=self.device)
        out_spike = torch.zeros(batch_size, self.output_dim, device=self.device)
        init_states_dict['out_layer'] = (out_spike, out_volt)
        # SNN
        output = self.snn(spike_data, init_states_dict, batch_size, spike_ts)
        return output

# Question 4: Encoding MNIST into spikes
Following is the function that converts an MNIST image into spikes. You have already implemented it using Numpy in Assignment 2. The implementation remains the same- except that we will now use Torch tensors instead of numpy arrays. Fill in the components to convert a batch of torch tensors into spikes. Since the goal is to learn writing optimized code using PyTorch, you are supposed to do this without any loops. Use vector operations instead. 

In [7]:
def img_2_event_img(image, device, spike_ts):
    """
    Transform image to event image
    Args:
        image (Tensor): image
        device (device): device (can be either CPU or GPU)
        spike_ts (int): spike timestep
    Returns:
        event_image: event image
    """
    batch_size = image.shape[0]
    channel_size = image.shape[1]
    image_size = image.shape[2]
    image = image.view(batch_size, channel_size, image_size, image_size, 1)
    
    #Create a random image of shape batch_size x channel_size x image_size x image_size x spike_ts. Torch rand function might be useful here. 
    #Remember to put the random image on the device specified in the function argument. 
    random_image = torch.rand(batch_size, channel_size, image_size, image_size, spike_ts)
    random_image = random_image.to(device)

    #Generate event image using image and random image
    event_image = (image > random_image)
    event_image = event_image.float()
    
    return event_image

# Question 5: Training the SNN

In this exercise, we will write the function to train an SNN using spatiotemporal backprop (stbp). A typical training loop works as follows:

1. Split the dataset into train and test. The network is trained on all the batches in the train dataset, and then validated on the test dataset. This gives us an idea of how well the network generalizes to unseen data. 

2. A criterion is defined to compute the loss. For classification tasks, this is generally Cross Entropy (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html)

3. The network is initialized with random weights. 

4. Typically, the training is done on mini-batches of train data. This means that at every training instance, the network receives batches of training data where each batch contains small number of samples. The loss and the gradients required for updating the weights are averaged over these batches. 

5. Within every training iteration, first we retrieve the batches of data. We compute the network prediction. Then we compute the loss by comparing the network prediction against the true labels. Then the gradient of the loss with respect to all the network weights is computed. This gradient is then used to update the weights in the network.  

Thankfully, PyTorch provides APIs to automate most of the above steps. Below is the function training an SNN that implements the algorithm presented above using PyTorch. Your task is to fill the components. Refer to the comments for hints on what PyTorch functions to use.   

In [8]:
predicted_tensor = None
l = None
def stbp_snn_training(network, spike_ts, device, batch_size=128, test_batch_size=256, epoch=100):
   
    """
    STBP SNN training
    Args:
        network (SNN): STBP learning SNN
        spike_ts (int): spike timestep
        device (device): device
        batch_size (int): batch size for training
        test_batch_size (int): batch size for testing
        epoch (int): number of epochs
    Returns:
        train_loss_list: list of training loss for each epoch
        test_accuracy_list: list of test accuracy for each epoch
    """
    
    #Creating folder where MNIST data is saved. Loading the MNIST dataset. This code is complete. Do not touch it. 
    try:
        os.mkdir("./data")
        print("Directory data Created")
    except FileExistsError:
        print("Directory data already exists")
    
    data_path = './data/'
    train_dataset = torchvision.datasets.MNIST(root=data_path, train=True, download=True,
                                               transform=transforms.ToTensor())
    test_dataset = torchvision.datasets.MNIST(root=data_path, train=False, download=True,
                                              transform=transforms.ToTensor())
    # Train and test dataloader
    #Given the train and test datasets, we need to create dataloaders to load the datasets in the right format. 
    #You can read about PyTorch Dataset and Dataloaders here: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
    #For now, this part is complete and you do not need to do anything here. 
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size,
                                  shuffle=False, num_workers=4)
    test_dataloader = DataLoader(test_dataset, batch_size=test_batch_size,
                                 shuffle=False, num_workers=4)

    # Next we need to define a criteria for computing the loss. 
    # Refer to https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html on how to define the cross entropy loss. 
    # Note that you just need to define the loss here (and not compute it) 
    
    
    criterion = nn.CrossEntropyLoss()

    #Define an optimizer that will perform the weight updates. You can find more about the optimizers in PyTorch here: https://pytorch.org/docs/stable/optim.html
    #Taking the help of the documentation above, create an optimizer for stochastic gradient descent (SGD). 
    optimizer = optim.SGD(network.parameters(), lr=0.1, momentum=0.9)
    

    # List for saving loss and accuracy
    train_loss_list, test_accuracy_list = [], []
    test_num = len(test_dataset)

    # Start training
    
    #Put the network on the device (typically GPU but can also be cpu)
    network.to(device)
    
    #Loop for the epochs
    for ee in range(epoch):
        #Keep track of running loss
        running_loss = 0.0
        running_batch_num = 0
        train_start = time.time()
        
        #Iterate over the training data in train dataloader
        for data in train_dataloader:
            #Retrieve the image and label from data
            image, label = data
          
            #Put the image and labels on the device
            image, label = image.to(device), label.to(device)
            
            #Convert images to event images
            event_image = img_2_event_img(image, device, spike_ts)

            #Before we backprop, we need to set the gradients for each tensor to zero. This is done using the zero_grad function in Pytorch
            optimizer.zero_grad()
            
            #Compute the network output for the event images
            output = network(event_image)
           
            #Compute the loss using the criterion defined previously. Store in a variable called loss
            loss = criterion(output.float(), label)     
            
            #Backpropagate the loss through the network. Use Pytorch backward() function: https://pytorch.org/docs/stable/generated/torch.Tensor.backward.html
            loss.backward()
            
            #Update the network weights by taking an optimizer 'step'. 
            #You can learn how to do that here: https://pytorch.org/docs/stable/optim.html#taking-an-optimization-step
            optimizer.step()
            
            #Updating tracking variables. Nothing to do here
            running_loss += loss.item()
            running_batch_num += 1

            
        train_end = time.time()
        train_loss_list.append(running_loss / running_batch_num)
        print("Epoch %d Training Loss %.4f" % (ee, train_loss_list[-1]), end=" ")
        
        
        #This ends one training iteration. After every training iteration, we can evaluate how well the network does on data that it has not seen before. 
        #This step is called testing and is done on test dataset. 
        
        #Counter to keep track of the number of correct predictions
        test_correct_num = 0
        test_start = time.time()
        with torch.no_grad():
            for data in test_dataloader:
                
                #Retrieve the image and label from test data
                image, label = data
                
                #Put the image and labels on the device
                image = image.to(device)
                label = label.to(device)

                #Convert the image into event images
                event_image = img_2_event_img(image, device, spike_ts)
                
                #Compute the network predictions and store in a variable called outputs
                outputs = network(event_image)
                
                #Get the class label as the largest activation. This is complete. Nothing to do here.
                _, predicted = torch.max(outputs, 1)

                #Compare the network predictions against the true labels and update the counter for correct predictions. No loops. 
                correct = (predicted == label).sum()
                test_correct_num += correct.item()
        
        #Updating tracking variables. Nothing to do here
        test_end = time.time()
        test_accuracy_list.append(test_correct_num / test_num)
        print("Test Accuracy %.4f Training Time: %.1f Test Time: %.1f" % (
            test_accuracy_list[-1], train_end - train_start, test_end - test_start))
    
    #Return the loss and accuracies. Nothing to do here. 
    print("End Training")
    network.to('cpu')
    return train_loss_list, test_accuracy_list

Now we have everything ready to train and test our SNN using backprop. All that is left to do is initialize the network with the right parameters and call the training function on it. 


In [9]:
# Define the device on which training will be performed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

torch.cuda.empty_cache() 

#Define the input dimensions in a variable
input_dim = 784

#Define the output dimensions in a variable
output_dim = 10

#Define the hidden layer dimension in a variable
hidden_dim = 256

#Create a dictionary of the neuron parameters for the hidden and output layer. The keys should be 'hid_layer' and 'out_layer'.
#The values of the dictionary should be a list of the neuron parameters for each layer where the list elements are [vdecay, vth, grad_win, grad_amp]
param_dict = {"hid_layer" : [0.5, 0.5, 0.5, 0.7], "out_layer" : [0.5, 0.5, 0.5, 0.7]}

#Define snn timesteps in a variable
spike_ts = 5

#Create the SNN using the class definition in 3b and the arguments defined above
network = WrapSNN(input_dim, output_dim, hidden_dim, param_dict, device)

#Define the following training parameters
#Batch size for training
batch_size = 128

#Batch size for testing
test_batch_size = 256

#Epochs
epoch = 50 

#Train the snn using the above arguments and the definition in Q5. 
train_loss_list, test_accuracy_list = stbp_snn_training(network, spike_ts, device, batch_size, test_batch_size, epoch)

Using cuda device
Directory data Created


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



...

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch 0 Training Loss 0.3863 Test Accuracy 0.9479 Training Time: 13.5 Test Time: 1.9
Epoch 1 Training Loss 0.2030 Test Accuracy 0.9569 Training Time: 13.7 Test Time: 1.9
Epoch 2 Training Loss 0.1650 Test Accuracy 0.9655 Training Time: 14.0 Test Time: 1.9
Epoch 3 Training Loss 0.1390 Test Accuracy 0.9637 Training Time: 13.8 Test Time: 1.9
Epoch 4 Training Loss 0.1278 Test Accuracy 0.9701 Training Time: 14.1 Test Time: 2.0
Epoch 5 Training Loss 0.1155 Test Accuracy 0.9701 Training Time: 13.9 Test Time: 2.1
Epoch 6 Training Loss 0.1092 Test Accuracy 0.9719 Training Time: 14.2 Test Time: 1.9
Epoch 7 Training Loss 0.1024 Test Accuracy 0.9709 Training Time: 13.0 Test Time: 1.9
Epoch 8 Training Loss 0.0966 Test Accuracy 0.9735 Training Time: 13.2 Test Time: 1.8
Epoch 9 Training Loss 0.0940 Test Accuracy 0.9746 Training Time: 13.4 Test Time: 1.9
Epoch 10 Training Loss 0.0896 Test Accuracy 0.9732 Training Time: 13.5 Test Time: 1.9
Epoch 11 Training Loss 0.0867 Test Accuracy 0.9778 Training Time

# Question 6: Concluding Remarks 
If you have been able to implement all the parts in Q.1.-Q.5., you can now feel proud of yourself for having learned the basics of a deep learning pipeline that powers all the modern AI applications. Of course, the exact implementation varies in the input and network design for more complex applications, but the pipeline remains the same. In fact, we went beyond the conventional deep learning, and implemented a spiking deep learning algorithm which is not a skill that many people possess. 

Now that you know the basic implementation, there are lots of directions which you can pursue if you are interested in this research area- how do the hyper-parameters such as the pseudo-gradient window affect the network training process? How much training data do you need to achieve good performance? How does the amount of training data vary with the complexity of the task? To what limits can you push the spike encoding, i.e. what is the minimum timesteps you need to achieve good performance? 

You do not need to answer these questions in this assignment but it is something to think about if you are interested. 

**As the concluding question of the course, can you describe what are the three key lessons you learned from the course and why is it important to learn them?**

## Answer 6
The three key lessons are as follows: <br>
1. The most important part of this course for me was backpropagation. I already knew this part but how Profesor taught this part from the basics of it was really helpful in understanding it properly. Backpropagation is very important in especially machine learning field. I think it is one of the main method to use in these neural networks. <br>
2. The programming part of this assignment was really important learning for me. The coding part was really hard and interesting to be honest. It was very important because it felt like I was practically implemeting the models rather than just learning them theoretically in the class. <br>
3. Lastly, I think the way Professor took lectures in the class was very good. I think If I am going to talk in front of people in future, then this way of giving lectures which makes it interesting for students to learn and clear all the doubts will be good learning to take. 